Este cuadernillo tiene como propósito cargar un archivo Excel que contiene múltiples hojas de datos clínicos y exportar cada hoja en un archivo CSV independiente. El procedimiento permite separar la información en tablas individuales y guardarlas en la carpeta de trabajo del proyecto. De esta manera, se asegura que cada dataset pueda ser utilizado de manera modular en los pasos posteriores del pipeline (prepro, join, check y exploration).

En este bloque se cargan las tablas originales de atención, triage, evolución e interconsultas. Se normalizan los tipos de datos y se verifican las dimensiones iniciales de cada dataset.

In [3]:
# Block 1 — Robust CSV loading (force dtypes, avoid DtypeWarning)

import pandas as pd
from typing import Dict, List

PATH = r"C:\Users\wilmerbelza\Documents\Prediction model"

# Columns we prefer as string (identifiers / text)
ID_LIKE_STR: List[str] = [
    'NumeroDocumento', 'TipoDocumento', 'Genero', 'Estado',
    'SalidaClinica', 'Observacion', 'Resultado', 'MotivoConsulta'
]
# Integer columns that may have missing values
INT_OPTIONAL: List[str] = ['Ingreso', 'Folio', 'ClasificacionTriage', 'EdadAtencion']

def build_dtype_map(csv_path: str) -> Dict[str, str]:
    """Inspect header and build dtype map: IDs->string, optional ints->Int64."""
    hdr = pd.read_csv(csv_path, nrows=0, low_memory=False)  # only header
    cols = list(hdr.columns)
    dtype_map: Dict[str, str] = {}
    for c in cols:
        if c in ID_LIKE_STR:
            dtype_map[c] = 'string'
        elif c in INT_OPTIONAL:
            dtype_map[c] = 'Int64'
    # If the first column is unnamed or mixed, force it to string
    if len(cols) > 0 and (cols[0] == '' or cols[0].startswith('Unnamed')):
        dtype_map[cols[0]] = 'string'
    return dtype_map

def read_csv_safe(csv_path: str) -> pd.DataFrame:
    """Read CSV using stable dtypes. Retry with a lighter map if needed."""
    dtype_map = build_dtype_map(csv_path)
    try:
        return pd.read_csv(csv_path, dtype=dtype_map, low_memory=False)  # default engine (C)
    except Exception:
        # Retry: keep only string dtypes for known ID-like columns
        slim_map = {k: v for k, v in dtype_map.items() if v == 'string'}
        return pd.read_csv(csv_path, dtype=slim_map, low_memory=False)

# Load raw tables
df_atencion  = read_csv_safe(f"{PATH}/atencion.csv")
df_triage    = read_csv_safe(f"{PATH}/triage.csv")
df_evolucion = read_csv_safe(f"{PATH}/evolucion.csv")
df_interc    = read_csv_safe(f"{PATH}/interconsultas.csv")

print("Atencion:", df_atencion.shape)
print("Triage:", df_triage.shape)
print("Evolucion:", df_evolucion.shape)
print("Interconsultas:", df_interc.shape)

# Quick dtype sanity
for name, df in {"atencion": df_atencion, "triage": df_triage, "evolucion": df_evolucion, "interc": df_interc}.items():
    print(f"\n[{name}] dtypes (sample):")
    print(df.dtypes.head(10))
    print(df.head(3))

Atencion: (63303, 9)
Triage: (57850, 10)
Evolucion: (684984, 11)
Interconsultas: (40999, 14)

[atencion] dtypes (sample):
NumeroDocumento          string[python]
TipoDocumento            string[python]
Genero                   string[python]
EdadAtencion                      Int64
Ingreso                           Int64
FechaAdmision                    object
FechaEgreso                      object
SalidaClinica            string[python]
FechaAnulacionIngreso            object
dtype: object
   NumeroDocumento TipoDocumento    Genero  EdadAtencion  Ingreso  \
0  01032023                  AS   Femenino            34        1   
1  01072711799               CC   Femenino            27        1   
2  0748972                   PA   Femenino            49        3   

         FechaAdmision          FechaEgreso SalidaClinica  \
0  2023-03-01 01:42:50  1753-01-01 00:00:00             N   
1  2023-08-17 18:15:08  2023-08-18 00:07:06             S   
2  2023-06-01 04:22:24  2023-06-02 07:46:57 

En este bloque se ajustan los nombres de columnas y se convierten los campos de fecha a formato datetime. Esto facilita las uniones posteriores y evita errores de consistencia.

In [5]:
# Block 2 — Normalize column names and dtypes

def normalize_dates(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors='coerce')
    return df

df_atencion = normalize_dates(df_atencion, ['FechaAdmision','FechaEgreso','FechaAnulacionIngreso'])
df_triage   = normalize_dates(df_triage,   ['FechaAdmision','FechaHoraAtencion'])
df_evolucion = normalize_dates(df_evolucion, ['FechaAdmision','FechaHoraAtencion','FechaEvolucion'])
df_interc    = normalize_dates(df_interc,    ['FechaAdmision','FechaHoraAtencion','FechaHoraOrden','FechaRespuesta','FechaCancelacion'])

En este bloque se filtran únicamente los pacientes con edad mayor o igual a 18 años, de modo que el pipeline se limite a la población adulta en coherencia con el estudio.

In [6]:
# Block 3 — Filter adults only (EdadAtencion >= 18)

df_atencion = df_atencion[df_atencion['EdadAtencion'] >= 18].copy()
df_triage = df_triage[df_triage['EdadAtencion'] >= 18].copy()
df_evolucion = df_evolucion[df_evolucion['EdadAtencion'] >= 18].copy()
df_interc = df_interc[df_interc['EdadAtencion'] >= 18].copy()

print("After filtering adults:")
print("Atencion:", df_atencion.shape)
print("Triage:", df_triage.shape)
print("Evolucion:", df_evolucion.shape)
print("Interconsultas:", df_interc.shape)

After filtering adults:
Atencion: (63303, 9)
Triage: (57850, 10)
Evolucion: (684984, 11)
Interconsultas: (40999, 14)


En este bloque se revisan duplicados y valores nulos en columnas críticas, con el fin de identificar posibles problemas antes de la unión de tablas.

In [7]:
# Block 4 — Initial QA checks

for name, df in {
    "atencion": df_atencion,
    "triage": df_triage,
    "evolucion": df_evolucion,
    "interconsultas": df_interc
}.items():
    print(f"\n=== {name.upper()} ===")
    print("Shape:", df.shape)
    print("Nulls (%):")
    print(df.isna().mean().round(3).sort_values(ascending=False).head(5))
    dups = df.duplicated(subset=['NumeroDocumento','Ingreso']).sum() if {'NumeroDocumento','Ingreso'}.issubset(df.columns) else 0
    print("Duplicated episodes:", dups)


=== ATENCION ===
Shape: (63303, 9)
Nulls (%):
NumeroDocumento    0.0
TipoDocumento      0.0
Genero             0.0
EdadAtencion       0.0
Ingreso            0.0
dtype: float64
Duplicated episodes: 26

=== TRIAGE ===
Shape: (57850, 10)
Nulls (%):
NumeroDocumento    0.0
TipoDocumento      0.0
Genero             0.0
EdadAtencion       0.0
Ingreso            0.0
dtype: float64
Duplicated episodes: 19

=== EVOLUCION ===
Shape: (684984, 11)
Nulls (%):
NumeroDocumento    0.0
TipoDocumento      0.0
Genero             0.0
EdadAtencion       0.0
Ingreso            0.0
dtype: float64
Duplicated episodes: 634333

=== INTERCONSULTAS ===
Shape: (40999, 14)
Nulls (%):
Observacion        0.653
Resultado          0.410
Genero             0.000
EdadAtencion       0.000
NumeroDocumento    0.000
dtype: float64
Duplicated episodes: 16923


En este bloque se exportan las tablas preprocesadas (con población filtrada y fechas normalizadas) para su posterior uso en la etapa de unión (join).

In [9]:
# Block 5 — Save cleaned datasets in main project folder

import os

OUT = r"C:\Users\wilmerbelza\Documents\Prediction model"
os.makedirs(OUT, exist_ok=True)  # create base directory if missing

def safe_to_csv(df, filename):
    path = os.path.join(OUT, filename)
    df.to_csv(path, index=False)
    print(f"Saved: {path}")

safe_to_csv(df_atencion,      "atencion_clean.csv")
safe_to_csv(df_triage,        "triage_clean.csv")
safe_to_csv(df_evolucion,     "evolucion_clean.csv")
safe_to_csv(df_interc,        "interconsultas_clean.csv")

print("\nAll preprocessed tables saved successfully.")

Saved: C:\Users\wilmerbelza\Documents\Prediction model\atencion_clean.csv
Saved: C:\Users\wilmerbelza\Documents\Prediction model\triage_clean.csv
Saved: C:\Users\wilmerbelza\Documents\Prediction model\evolucion_clean.csv
Saved: C:\Users\wilmerbelza\Documents\Prediction model\interconsultas_clean.csv

All preprocessed tables saved successfully.
